In [ ]:
# 1. 데이터 로드
# pdf ver.
from langchain_community.document_loaders import PyMuPDFLoader

pdf_paths = [
            "저작권법(법률)(제20358호)(20240828).pdf", 
            "저작권상담사례집2024 (3).pdf", 
            "인공지능과 저작권 제1-2부.pdf", 
            "최진원_알기 쉬운 저작권 계약 가이드북(제2판)_2024.pdf", 
            "naver.pdf",
            "1인 미디어 창작자를 위한 저작권 안내서(2019).pdf",
            "US_copyright.pdf",
            "wipo_copyright.pdf",
            "공공저작물 저작권 관리 및 이용 지침 해설서(개정20240101업로드용).pdf",
            "네이버 블로그.pdf",
            "카카오 서비스 약관20230109.pdf",
            "하버드)해외 저작권, 공정이용 가이드라인.pdf",
            "생성형AI 저작권 가이드라인.pdf"
            ]

import fitz  # PyMuPDF

# markdown에 넣기
def pdfs_as_markdown_blocks(pdf_paths: list[str]) -> list[dict]:
    all_blocks = []

    for pdf_path in pdf_paths:
        doc = fitz.open(pdf_path)
        for page in doc:
            blocks_data = page.get_text("dict")["blocks"]
            for block in blocks_data:
                if "lines" not in block:
                    continue
                text = ""
                for line in block["lines"]:
                    for span in line["spans"]:
                        text += span["text"]
                text = text.strip()
                if text:
                    # 마크다운 구조 분류
                    if text.startswith("제") and "조" in text:
                        all_blocks.append({"type": "section", "text": f"## {text}"})
                    elif text.endswith("가이드") or len(text) < 20:
                        all_blocks.append({"type": "title", "text": f"# {text}"})
                    else:
                        all_blocks.append({"type": "paragraph", "text": text})
    return all_blocks

In [2]:
all_blocks = pdfs_as_markdown_blocks(pdf_paths)

In [3]:
all_blocks[5]

{'type': 'title', 'text': '# 제1장 총칙'}

In [3]:
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 1. Markdown header 기반 splitter 정의
header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "title"), ("##", "section")])

# 2. 블록 → Markdown 텍스트로 변환
def blocks_to_markdown_text(blocks):
    return "\n\n".join(b["text"] for b in blocks)

markdown_text = blocks_to_markdown_text(all_blocks)

# 3. Markdown header 기준으로 구조 단위로 분할 (Document 객체 반환됨)
structured_chunks = header_splitter.split_text(markdown_text)

# 4. Recursive splitter 설정
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)

# 5. 구조 단위 chunk들을 다시 세부적으로 쪼개고, metadata 유지
final_split_docs = []
for doc in structured_chunks:
    content = doc.page_content      # ✅ Document 객체로부터 텍스트 추출
    metadata = doc.metadata         # ✅ Document 객체로부터 메타데이터 추출

    small_chunks = recursive_splitter.split_text(content)
    for chunk in small_chunks:
        final_split_docs.append(Document(page_content=chunk, metadata=metadata))


In [4]:
len(final_split_docs)

5623

In [16]:
final_split_docs[3]

Document(metadata={'title': '제1장 총칙', 'section': '제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2009. 4. 22., 2011. 6. 30., 2011. 12. 2., 2016. 3. 22.,'}, page_content='2021. 5. 18., 2023. 8. 8.>  \n1. “저작물”은 인간의 사상 또는 감정을 표현한 창작물을 말한다.  \n2. “저작자”는 저작물을 창작한 자를 말한다.  \n3. “공연”은 저작물 또는 실연(實演)ㆍ음반ㆍ방송을 상연ㆍ연주ㆍ가창ㆍ구연ㆍ낭독ㆍ상영ㆍ재생 그 밖의 방법으로  \n공중에게 공개하는 것을 말하며, 동일인의 점유에 속하는 연결된 장소 안에서 이루어지는 송신(전송은 제외한다')

In [5]:
from langchain_core.documents import Document

def assign_metadata(docs: list[Document]) -> list[Document]:
    result = []

    for doc in docs:
        text = doc.page_content.lower()
        metadata = doc.metadata.copy()  # 기존 metadata 보존

        # 1. 플랫폼
        platforms = ["네이버", "카카오", "유튜브", "인스타그램", "naver", "kakao", "youtube", "instagram"]
        matched_platforms = [p for p in platforms if p in text]
        if matched_platforms:
            metadata["platform"] = ", ".join(matched_platforms)  # ✅ 리스트를 문자열로 변환

        # 2. 법 영역 (국내/해외)
        if any(word in text for word in ["fair use", "dmca", "united states", "미국"]):
            metadata["law_scope"] = "해외"
        elif any(word in text for word in ["저작권법", "공공누리", "kogl", "대한민국"]):
            metadata["law_scope"] = "국내"

        # 3. 문서 유형
        if any(word in text for word in ["사례", "faq"]):
            metadata["doc_type"] = "사례집"
        elif any(word in text for word in ["가이드", "guide"]):
            metadata["doc_type"] = "가이드"
        elif any(word in text for word in ["법", "조항", "제"]):
            metadata["doc_type"] = "법령"

        # 4. 출처
        if "저작권법" in text:
            metadata["source"] = "저작권법"
        elif "dmca" in text:
            metadata["source"] = "DMCA"
        elif "공공누리" in text or "kogl" in text:
            metadata["source"] = "KOGL"
        elif "크리에이티브 커먼즈" in text or "creative commons" in text:
            metadata["source"] = "CC"

        # 5. 토픽 자동 태깅
        keyword_to_topic = {
            "음악": "음악사용",
            "배경음악": "음악사용",
            "이미지": "이미지사용",
            "ai": "ai저작권",
            "인공지능": "ai저작권",
            "공정이용": "공정이용",
            "인용": "인용",
            "계약": "저작권계약",
            "저작권료": "저작권계약",
            "공공저작물": "공공저작물"
        }
        topics = {tag for keyword, tag in keyword_to_topic.items() if keyword in text}
        if topics:
            metadata["topic"] = ", ".join(topics)  # ✅ 리스트 → 문자열 변환

        # 문서에 metadata 반영
        doc.metadata = metadata
        result.append(doc)

    return result


In [6]:
result_docs = assign_metadata(final_split_docs)
result_docs 

[Document(metadata={'doc_type': '법령'}, page_content='법제처                                                            1                                                       국가법령정보센터'),
 Document(metadata={'title': '저작권법', 'doc_type': '법령'}, page_content='[시행 2024. 8. 28.] [법률 제20358호, 2024. 2. 27., 일부개정]  \n문화체육관광부 (저작권정책과) 044-203-2477'),
 Document(metadata={'title': '제1장 총칙', 'section': '제1조(목적) 이 법은 저작자의 권리와 이에 인접하는 권리를 보호하고 저작물의 공정한 이용을 도모함으로써 문화 및'}, page_content='관련 산업의 향상발전에 이바지함을 목적으로 한다. <개정 2009. 4. 22.>'),
 Document(metadata={'title': '제1장 총칙', 'section': '제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2009. 4. 22., 2011. 6. 30., 2011. 12. 2., 2016. 3. 22.,', 'doc_type': '법령'}, page_content='2021. 5. 18., 2023. 8. 8.>  \n1. “저작물”은 인간의 사상 또는 감정을 표현한 창작물을 말한다.  \n2. “저작자”는 저작물을 창작한 자를 말한다.  \n3. “공연”은 저작물 또는 실연(實演)ㆍ음반ㆍ방송을 상연ㆍ연주ㆍ가창ㆍ구연ㆍ낭독ㆍ상영ㆍ재생 그 밖의 방법으로  \n공중에게 공개하는 것을 말하며, 동일인의 점유에 속하는 연결된 장소 안에서 이루어지는 송신(전송은 제외한다'),
 Document(metadata={'title': ')을 포함한다.', 'doc_type': '법령'}, page_conte

In [7]:
print(len(result_docs))  # ✅ 최소 1 이상이어야 함

5623


In [25]:
result_docs[6]

Document(metadata={'title': '이터베이스를 포함한다.', 'doc_type': '법령'}, page_content='18. “편집저작물”은 편집물로서 그 소재의 선택ㆍ배열 또는 구성에 창작성이 있는 것을 말한다.  \n법제처                                                            2                                                       국가법령정보센터')

In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
# 3. embedding 모델 생성 
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

In [10]:
filtered_docs = [doc for doc in result_docs if doc.page_content.strip() != ""]
print(f"✅ page_content 있는 문서 수: {len(filtered_docs)}")

✅ page_content 있는 문서 수: 5623


In [11]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
import math

# 1. Chroma 인스턴스 생성
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name="rag_chatbot",
    persist_directory="vector_store/chroma/rag_chatbot"
)

# 2. 문서 배치 추가 함수 정의
def batch_add_documents(vector_store, documents: list[Document], batch_size: int = 500):
    total = len(documents)
    num_batches = math.ceil(total / batch_size)

    for i in range(num_batches):
        batch = documents[i * batch_size : (i + 1) * batch_size]
        vector_store.add_documents(batch)
        print(f"✅ Added batch {i+1}/{num_batches} (size: {len(batch)})")

# 문서 추가
batch_add_documents(vector_store, result_docs, batch_size=500)

# # 저장 메시지 출력 (persist 호출 없이)
# print("✅ Vector store saved to disk at:", persist_directory)

✅ Added batch 1/12 (size: 500)
✅ Added batch 2/12 (size: 500)
✅ Added batch 3/12 (size: 500)
✅ Added batch 4/12 (size: 500)
✅ Added batch 5/12 (size: 500)
✅ Added batch 6/12 (size: 500)
✅ Added batch 7/12 (size: 500)
✅ Added batch 8/12 (size: 500)
✅ Added batch 9/12 (size: 500)
✅ Added batch 10/12 (size: 500)
✅ Added batch 11/12 (size: 500)
✅ Added batch 12/12 (size: 123)


In [12]:
# 검색 설정들을 넣어서 retriever 생성 -
retriever= vector_store.as_retriever()
#     search_type = "similarity_score_threshold",  # "similarity"(기본), "similarity_score_threshold"->임계치 설정, "mmr" -> 비율 정의
#     search_kwargs =  {"k": 5, "score_threshold": 0.5}  #유사도 점수가 지정한 값 이상인 것만 조회 #검색 메소드(similarity_search_XXXXX())의 파라미터 전달
# )
result_docs = retriever.invoke("네이버 관련 저작권을 설명해줘")
result_docs

[Document(id='418c614a-152f-490c-adf5-94c949b3ed4a', metadata={'law_scope': '국내', 'source': '저작권법', 'topic': '이미지사용, 저작권계약', 'title': '태그', 'platform': '네이버', 'doc_type': '법령'}, page_content="저작권이 침해 당한 경우 어떻게 신고하나요?  \n네이버 내 등록된 게시물 내에 본인의 저작물(게시글 본문, 이미지, 동영상 등)이 허락없이 포함된 경우 권리 침해 신고를 통해 해당 게시물에 대한 게시 중단을 요청할 수 있습니다.  \n권리보호센터 상단의 '권리 침해 신고' 페이지로 접속하셔서 본인인증 절차 후 저작권 신고를 진행해 주세요.  \n저작권 침해 사유로 신고를 접수하시는 경우 본인이 해당 저작물의 저작권자임을 확인할 수 있는 증빙자료를 함께 접수해 주셔야 합니다. 보다 자세한 사항은 필요 서류 안내를 참고해 주세요.  \n저작권 침해로 신고 가능한 대상은 무엇인가요?  \n네이버 내에 등록된 게시물로 인해 저작권 또는 그 밖에 저작권법에 따라 보호되는 권리를 침해받고 있는 경우 저작권 침해 사유로 권리 침해 신고를 접수하실 수 있습니다.  \n권리 침해 신고 시에는 네이버 쇼핑 서비스를 통해 확인 가능한 상품 페이지와 카페, 블로그 등 다른 사람이 작성한 게시글을 구분하여 접수해 주세요.  \n저작권 침해 신고 시 필요한 서류는 무엇인가요?  \n우선 신고자가 저작권리자임을 확인할 수 있는 증빙 자료를 준비해 주세요. 저작권리자임을 증빙하는 자료는 다음과 같습니다.  \n1. 자신이 그 저작물 등의 권리자로 표시된 저작권 등의 등록증 사본 또는 그에 상당하는 자료  \n2. 자신의 성명 등 또는 널리 알려진 이명이 표시되어 있는 그 저작물 등의 사본 또는 그에 상당하는 자료  \n3. 저작권 등을 가지고 있는 자로부터 적법하게 복제·전송의 허락을 받은 사실을 증명하는 계약서 사본 또는 그에"),
 Documen

In [13]:
# retriever | 질문과 검색 결과를 프롬프트로 생성 | LLM | Output => Chain
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document


# 가상 답변 문서

hyde_model = ChatOpenAI(model_name = "gpt-4.1")
hyde_prompt_template = PromptTemplate(
    template = """#Instruction:
다음 질문에 대해서 완전하고 상세한 답변으로 실제 사실에 기반해서 작성해주세요.
질문과 관련된 내용만으로 작성합니다.
답변과 직접적인 연관성이 없는 내용은 답변에 포함시키지 않습니다.

# 질문:
{query}

"""
)
hyde_chain = hyde_prompt_template | hyde_model | StrOutputParser()


# 최종 chain: hyde_chain | 답변처리 chain
# 답변 처리 chain
answer_model = ChatOpenAI(model_name="gpt-4.1-mini")
template = """# Instruction:
당신은 정확한 정보 제공을 우선시하는 인공지능 어시스턴트입니다.
주어진 Context에 포함된 정보만 사용해서 질문에 답변하세요.
Context에 질문에 대한 명확한 정보가 있는 경우 그 내용을 바탕으로 답변하세요.
Context에 질문에 대한 명확한 정보없을 경우 "정보가 부족해서 답을 알 수 없습니다." 라고 대답합니다.
절대 Context에 없는 내용을 추측하거나 일반 상식을 이용해 답을 만들어서 대답하지 않습니다.

# Context:
{context}

# 질문:
{query}
"""
answer_prompt_template = PromptTemplate(template=template)

answer_chain = answer_prompt_template | answer_model | StrOutputParser()


###############################
# 최종 chain = hyde_chain + answer_chain


def format_docs(docs:list[Document]) -> str:
    """
    Retriever가 검색한 문서들에서 page_content(문서 내용)만 추출해서 반환.
    추출된 문서의 내용들을 "\n\n"로 연결한다.
    Args:
        docs(list[Document]) -검색한 문서 리스트
    Returns:
        str - 문서1내용+\n\n문서2내용+\n\n ..
    """
    return "\n\n".join(doc.page_content for doc in docs)

# answer_chain은 context와 query를 받음/ hydechain은 query를 받음. 근데 answer_chain을 넘겨주려면 
# context가 retriever되고 그걸 리스트(format_docs)로 보내줘야 하니깐 아래와 같은 체인을 만든다.
final_chain = ({"context": hyde_chain| retriever | format_docs, "query":lambda x: x['query']}
            |answer_chain)

In [14]:
query = "유튜브 저작권에 대해 알려줘."
response = final_chain.invoke({"query": query})

print(response)

유튜브 저작권에 관한 내용은 다음과 같습니다.

- 유튜브의 콘텐트 아이디(Content ID)는 아무나 사용할 수 있는 것이 아니며, 유튜브가 제시하는 기준에 부합하는 저작권 소유자만 사용 승인을 받을 수 있습니다. 구체적으로, 유튜브 창작자 커뮤니티에서 자주 업로드되는 원본 자료의 상당 부분에 대해 독점권을 보유한 저작권자에게만 Content ID가 부여됩니다.

- 저작권자가 저작물을 제출하면 이후 올라오는 동영상은 Content ID 데이터베이스와 자동으로 비교되어 검사됩니다. 검사 결과에 따라 차단, 광고 수익 공유 등 사전에 설정된 옵션에 따라 처리됩니다.

- 유튜브에서 발생하는 광고 수익의 일부를 플랫폼이 가져가는 것은 기본적으로 사용 비용이며, 저작권료와는 별개입니다. 다만, Content ID 옵션 중 권리자에게 광고 수익을 전달하는 경우도 있습니다.

- 영상 중 일부에 저작권 침해가 발생하여 영상 전체가 차단된 경우, 문제되는 부분을 삭제하거나 음소거하는 등 편집을 하면 재게시가 가능합니다. 유튜브 스튜디오에서 오디오 음소거, 노래만 음소거 등의 편집 툴이 제공됩니다.

- 저작권 문제 해결을 위해 플랫폼에 사전 보호 요청을 할 수 있으며, 유튜브도 Content ID 시스템을 통해 저작권 침해를 자동으로 감지하고 처리합니다.

이와 같이 유튜브 저작권 시스템은 저작권자를 보호하기 위해 자동 검사·차단 시스템(Content ID)을 운영하며, 저작권 침해 발생 시 편집을 통해 문제 부분만 수정하여 재게시할 수 있도록 지원하고 있습니다.
